<a href="https://colab.research.google.com/github/khushbooG9/Social-Media-Mining/blob/master/community_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Collabrated with Shakti Priya and Dinesh
import json
import numpy as np
import pandas as pd
import operator
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.annotations import LabelSet
from bokeh.models.graphs import from_networkx
import matplotlib.pyplot as plt
# Though the following import is not directly being used, it is required
# for 3D projection to work
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn import datasets

In [0]:
import networkx as nx

In [6]:

!pip install twitter

     |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
from twitter import *

In [0]:
# To make it more readable, lets store
# the OAuth credentials in strings first.
CONSUMER_KEY = '############'
CONSUMER_SECRET = '############'
OAUTH_TOKEN = '	#################'
OAUTH_TOKEN_SECRET = '###########################'
    
# Then, we store the OAuth object in "auth"
auth = OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

# Notice that there are four tokens - you need to create these in the
# Twitter Apps dashboard after you have created your own "app".

# We now create the twitter search object.
t = Twitter(auth=auth)


In [0]:
import sys
import time
from twitter.api import TwitterHTTPError
from urllib.error import URLError
from http.client import BadStatusLine

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
        if wait_period > 3600: # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e
        if e.e.code == 401:
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429:
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered %i Error. Retrying in %i seconds' % (e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function

    wait_period = 2
    error_count = 0
    while True:
        try:
            return twitter_api_func(*args, **kw)
        except TwitterHTTPError as e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise

In [0]:
# Create a mostly empty data frame,
# and write it to a CSV file.
df = pd.DataFrame(columns=['ID','followers'])
df.to_csv('friend9.csv', index=False)

# Our function
def save_followers(fid, followers):
    df = pd.DataFrame([[fid, followers]], columns=['ID','followers'])
    with open('friend9.csv', 'a') as f:
        df.to_csv(f,header=False, index=False)

In [0]:
def crawl_followers(twitter_api, screen_name, limit=1000000, depth=2):
    
    # Resolve the ID for screen_name and start working with IDs for consistency
    print("level 1 reciprocal friends:")
    seed_id = str(twitter_api.users.show(screen_name=screen_name)['id'])
    response = make_twitter_request(t.friends.ids,
                                user_id=seed_id, count = 5000)
    friends = response["ids"]
    # Getting followers' ids
    response = make_twitter_request(t.followers.ids,
                                user_id=seed_id, count = 5000)
    followers = response["ids"]
    # Computing reciprocal friends
    reciprocal_friends = set(friends) & set(followers)
    rfriends = []
    for friend in reciprocal_friends:
        #print(friend)
        response = make_twitter_request(t.users.show,user_id=friend)
        rfriends.append((response.get('id'),response.get('followers_count')))
    rfriends.sort(key=operator.itemgetter(1),reverse=True)
    next_queue=[x[0] for x in rfriends[:5]]
    print(next_queue)
    print("found 5 rec friends with max followers and saving it to csv")
    # Store a seed_id => _follower_ids mapping in MongoDB
    save_followers(seed_id, ','.join([str(x) for x in next_queue]))
    
    d = 1
    
    while d < depth:
        d += 1
        
        (queue, next_queue) = (next_queue, [])
        # Loop through the current
        # level of followers
        print("level %d starts." % (d))
        for fid in queue:
            response = make_twitter_request(t.friends.ids,
                                user_id=seed_id, count = 5000)
            friends = response["ids"]
            # Getting followers' ids
            response = make_twitter_request(t.followers.ids,
                                user_id=seed_id, count = 5000)
            followers = response["ids"]
            # Computing reciprocal friends
            reciprocal_friends = set(friends) & set(followers)
            # IDs of followers of the user with ID "fid"
            rqueue = set(friends) & set(followers)
            dfriends = []
            for friend in reciprocal_friends:
                #print(friend)
                response = make_twitter_request(t.users.show,user_id=friend)
                dfriends.append((response.get('id'),response.get('followers_count')))
                dfriends.sort(key=operator.itemgetter(1),reverse=True)
            rqueue=[x[0] for x in dfriends[:5]]
            print("found 5 rec friends of %d with max followers and saving it to csv" % (fid))
            save_followers(str(fid), ','.join([str(x) for x in rqueue]))
            # Extending the list
            next_queue += rqueue


In [0]:
screen_name="ZedShaw"
crawl_followers(t, screen_name, depth=5, limit=5000)

In [0]:
import csv
G=nx.Graph()
with open('friend9.csv','r') as csv_file:
    csvread = csv.reader(csv_file)
    next(csvread)
    for line in csvread:
        if line[0] not in G:
            G.add_node(line[0])
            for cnodes in line[1].split(","):
                if cnodes not in G:
                    G.add_node(cnodes)
                G.add_edge(line[0],cnodes)
        else:
            for cnodes in line[1].split(","):
                if cnodes not in G:
                    G.add_node(cnodes)
                G.add_edge(line[0],cnodes)   

In [0]:
nx.draw(G,node_size=80)

In [0]:
k_comm_3 = list(nx.community.k_clique_communities(G,3))
k_comm_3


In [0]:
nodelist=list(G.nodes())
colorlist=[]
for x in range(0,len(nodelist)):
    colorlist.append(0.1)
color=0.9;
for itr in k_comm_3:
    for i in itr:
        colorlist[nodelist.index(i)]=color
    color-=0.2

In [0]:
nx.draw(G, node_size=200 , node_color=colorlist)